This code gets the adjusted networks necessary for displaying tanglegram figures as well as for conducting their relevant analyses.

### Reducing a network to clades

In [5]:
include(joinpath(@__DIR__, "..", "..", "helpers/set-seed.jl"))
include(joinpath(@__DIR__, "..", "..", "helpers/file-locking.jl"))
include(joinpath(@__DIR__, "..", "..", "helpers/pipelines.jl"))
include(joinpath(@__DIR__, "..", "..", "helpers/save-results.jl"))
include(joinpath(@__DIR__, "..", "..", "helpers/data-simulation.jl"))
include(joinpath(@__DIR__, "..", "..", "helpers/est_constraints-helpers.jl"))
include(joinpath(@__DIR__, "..", "..", "helpers/estimated_gts-helpers.jl"))
include(joinpath(@__DIR__, "..", "..", "helpers/pruned_simulation-helpers.jl"))
include(joinpath(@__DIR__, "..", "..", "helpers/results-getters.jl"))

get_completed_params (generic function with 1 method)

In [24]:
ntaxa = 1000; rep = 16; ils = "low"; ngt = 1000; m = 20;
# rep = 93;

truenet = load_true_net_ils_adjusted(ntaxa, rep, ils);
mnet = get_mnet(ntaxa, rep, ils, ngt, m);
_, mnet = find_minimum_retic_subset_hwcd_greedy(truenet, mnet, verbose=true, swaponerror=true);
for e in truenet.edge e.length = -1.0; e.gamma = -1.0; end

InterruptException: InterruptException:

In [25]:
println(writeTopology(truenet))

(((((((((t408,t409),((t413,(t420,t421)),(((t406,(t415,t416)),((t405,(((t418,t419),t417),t407)),((t410,t411))#H5)),(t412,(((t422,t423),(t424,t425)),t414))))),t402),((t403,(t404,#H5)),t401)),((((((t582,t583),((t594,t595),(t599,t600))),(t596,#H7)),((t576,t577),((t579,(t587,t588)),(t585,t586)))),((((t597,(t598)#H7),(t589,t590)),t584),((t578,(t580,t581)),(t591,(t592,t593))))),(((((t624,t625),(t620,t621)),(t618,t619)),(((t602)#H8,((t613,(t616,t617)),t611)),((t610,((t614,t615),(t612,(t622,t623)))),(t606,t607)))),((t601,(t608,t609)),((t604,t605),(t603,#H8)))))),((((t552,t553),(t551,(((t562,t563),t558),(t560,(t572,t573))))),((((t557,(t568,t569)),t555),(t554,(t574,t575))),(((t564,t565),((t570,t571),t559)),((t561,(t566,t567)),t556)))),((((((((t815,t816),t808),t807),(t817,t818)),(t804,(t823,(t824,t825)))),(((t811,t812),(((t813,t814),t806),t805)),t801)),(((t821,t822),(t819,t820)),(t802,((t809,t810),t803)))),((t830,t831),(((t828,(t840,t841)),(t827,((t844,(t845,t846)),t839))),(((((t835,t836),t829),(t

In [23]:
println(writeTopology(mnet))

(OUTGROUP,(((((((((((t100,t99),t94),(t95,t96)),t89),t78),((t82,t83),((t85,t86),(t90,t91)))),(((t76,(t84,(t97,t98))),((t77,(t92,t93)),(t79,(t87,t88)))),(t80,t81))),(((((((((t1000,t999),t993),((t985,t986),(t992,(t994,t995)))),((t979,((t983,t984),(t989,(t996,(t997,t998))))),((t980,(t990,t991)),((t981,t982),(t987,t988))))),(t977,t978)),t976),(((t951,((t952,(t964,t965)),((t959,t960),(t962,t963)))),(((t953,(t961,(t968,t969))),((t957,t958),(t974,t975))),((t954,(t956,(t966,t967))),(t970,t971)))),(t955,(t972,t973)))),(((t301,((t313,t314),(t322,t323))),(((t308,(t311,t312)),(t315,(t320,t321))),(t309,(t318,t319)))),((((t306,(t307,(t310,(t316,t317)))),(t303,#H5:::0.16408800903559995)),((t304,t305),(t324,t325))),(t302)#H5:::0.8359119909644))),((((t501,(t524,t525)),(((t502,(t504,t505)),(t503,((t510,(t513,t514)),(t522,t523)))),((t511,t512),(t516,t517)))),((t506,(t515,(t520,t521))),((t507,t508),(t509,(t518,t519))))),(((t526,((t529,(t539,t540)),(t533,(t545,t546)))),(t528,((t530,(t547,t548)),((t531,t532)

Helper functions

In [3]:
function trim_to_clades!(net::HybridNetwork, clade_size::Int)
    to_remove = [];
    for leaf in net.leaf
        if leaf.name == "OUTGROUP" continue end
        leaf_num = parse(Int, leaf.name[2:length(leaf.name)])
        if leaf_num % clade_size == 0
            leaf.name = "clade$(Int(leaf_num / clade_size))"
            continue
        end
        push!(to_remove, leaf.name)
    end

    for leaf_name in to_remove
        deleteleaf!(net, leaf_name)
    end
end

function gather_clades(net::HybridNetwork, clade_size::Int)
    clades = [Vector{String}() for _ = 1:(net.numTaxa ÷ clade_size)]
    for (i, leaf) in enumerate(net.leaf)
        if leaf.name == "OUTGROUP" continue end
        leaf_num = parse(Int, leaf.name[2:length(leaf.name)])
        push!(clades[((leaf_num -1) ÷ clade_size) + 1], leaf.name)
    end
    return pruneTruthFromDecomp(net, clades)
end

gather_clades (generic function with 1 method)

### Get the networks reduced down to their clades

In [14]:
clade_size = 2;

adj_mnet = deepcopy(mnet);
adj_true = deepcopy(truenet);

trim_to_clades!(adj_mnet, clade_size);
trim_to_clades!(adj_true, clade_size);

In [15]:
println(writeTopology(adj_mnet))

(OUTGROUP,(((((((clade39,((clade47,clade50),clade48)),((clade43,clade45),clade41)),(((clade38,(clade42,clade49)),(clade46,clade44)),clade40)),(((((((((clade496,clade497),clade493),clade500),(((clade490,clade495),(clade491,clade494)),(clade492,(clade498,clade499)))),clade489),clade488),(((((clade479,clade487),clade484),((clade477,(clade478,clade483)),clade485)),((clade476,clade482),(clade480,clade481))),clade486)),(((((clade154,clade156),clade160),clade159),(clade157,clade161)),((((clade153,(clade155,clade158)),#H5:::0.0005510855792618444),(clade152,clade162)),(clade151)#H5:::0.9994489144207381))),((((((clade251,clade252),((clade255,clade257),clade261)),(clade256,clade258)),clade262),((clade253,clade260),(clade254,clade259))),(((clade263,(clade270,clade273)),(clade264,((clade265,clade274),(clade266,clade267)))),(((clade268,clade271),(clade269,clade275)),clade272))))),((((((clade55,clade61),((clade53,(clade54,(clade56,clade57))),(clade58,clade60))),((clade51,clade59),clade62)),clade52),(

In [16]:
println(writeTopology(adj_true))

((((((((((((clade203:3.5297707860933416,clade208:3.5297707860933447):0.9178517747335968,((clade205:3.8898335106750297)#H5:0.0::0.8303600717746086,clade209:5.000859826596036):0.65467159207646):0.7044947425255247,(clade206:3.280199690701504,((clade211:1.7626480583138475,clade212:1.818108614888382):0.9197187630674931,clade207:2.288884337135442):0.9913153535660653):1.8719176126509591):0.5363367179900154,clade210:5.4256348470490074):0.5759743414160008,clade204:6.001609188011088):0.5363367179900154,clade201:5.363367179899911):2.4139823719731925,(clade202:3.792950968750482,#H5:0.5363367179900154::0.16963992822539142):5.258307915811049):0.9212780828439053,((((((clade297:1.4529183088727646,clade300:1.5379304203166178):3.080580619449852,clade291:4.108354293089013):0.5363367179900154,(clade298:0.5363367179897525,#H7:0.5363367179900154::0.16478028515079735):4.3158940594097475):2.285771630312757,((clade294:4.591556550502099,clade293:4.591556550956016):2.027896844689975,clade288:6.619453393376395):0

### Error calculations

In [11]:
@info "Reduce network rooted error: $(hardwiredClusterDistance(adj_mnet, adj_true, true))"

mnet_clades = gather_clades(mnet, clade_size);
true_clades = gather_clades(truenet, clade_size);

@info "Sum of clade error: $(sum(hardwiredClusterDistance(mc, tc, false) for (mc, tc) in zip(mnet_clades, true_clades)))"

┌ Info: Reduce network rooted error: 4
└ @ Main /mnt/dv/wid/projects4/SolisLemus-network-merging/InPhyNet-Simulations/est-gts/analysis/tanglegram.ipynb:1
┌ Info: Sum of clade error: 10
└ @ Main /mnt/dv/wid/projects4/SolisLemus-network-merging/InPhyNet-Simulations/est-gts/analysis/tanglegram.ipynb:6


### Next steps

Open SplitsTree, load the adjusted mnet and true nets in, then on TreeView click `Tree > Show Tanglegram`

# Zoom in on errors figure

In [10]:
mnet_desc = [sort(unique([d.name for d in get_descendant_leaves(n)])) for n in mnet.node];
true_desc = [sort(unique([d.name for d in get_descendant_leaves(n)])) for n in truenet.node];
diff_desc = unique(symdiff(mnet_desc, true_desc));
relevant_clades = [];
for desc_i in diff_desc
    any_matches = false
    for desc_j in diff_desc
        if desc_i == desc_j continue end
        if all(e -> e in desc_j, desc_i)
            any_matches = true
            break
        end
    end
    if !any_matches
        push!(relevant_clades, desc_i)
    end
end
relevant_clades

3-element Vector{Any}:
 ["t401", "t403", "t404", "t410", "t411"]
 ["t427", "t428", "t432", "t435", "t436", "t442", "t443"]
 ["t301", "t302", "t308", "t309", "t311", "t312", "t313", "t314", "t315", "t318", "t319", "t320", "t321", "t322", "t323"]

In [20]:
mnet_clusters = [cluster[2:1002] for cluster in eachrow(hardwiredClusters(mnet, tipLabels(mnet)))]
true_clusters = [cluster[2:1002] for cluster in eachrow(hardwiredClusters(truenet, tipLabels(mnet)))]
diff_clusters = symdiff(mnet_clusters, true_clusters)
idxs = [findall(c .== 1) for c in diff_clusters]
[tipLabels(mnet)[idx] for idx in idxs]

7-element Vector{Vector{String}}:
 ["t401", "t403", "t404"]
 ["t403", "t404"]
 ["t401", "t403", "t404", "t410", "t411"]
 ["t403", "t404", "t410", "t411"]
 ["t404", "t410", "t411"]
 ["t428", "t432", "t436", "t435", "t427", "t442", "t443"]
 ["t301", "t313", "t314", "t322", "t323", "t308", "t311", "t312", "t315", "t320", "t321", "t309", "t318", "t319", "t302"]

Next step: in the rendered PDFs/SVGs for mnet, look for the clade (`CTRL+F`) that contains a given vector of names and find its corresponding match in the truenet. They should differ *somehow*

In [21]:
println(writeTopology(mnet))

(OUTGROUP,(((((((((((t100,t99),t94),(t95,t96)),t89),t78),((t82,t83),((t85,t86),(t90,t91)))),(((t76,(t84,(t97,t98))),((t77,(t92,t93)),(t79,(t87,t88)))),(t80,t81))),(((((((((t1000,t999),t993),((t985,t986),(t992,(t994,t995)))),((t979,((t983,t984),(t989,(t996,(t997,t998))))),((t980,(t990,t991)),((t981,t982),(t987,t988))))),(t977,t978)),t976),(((t951,((t952,(t964,t965)),((t959,t960),(t962,t963)))),(((t953,(t961,(t968,t969))),((t957,t958),(t974,t975))),((t954,(t956,(t966,t967))),(t970,t971)))),(t955,(t972,t973)))),(((t301,((t313,t314),(t322,t323))),(((t308,(t311,t312)),(t315,(t320,t321))),(t309,(t318,t319)))),((((t306,(t307,(t310,(t316,t317)))),(t303,#H5:::0.16408800903559995)),((t304,t305),(t324,t325))),(t302)#H5:::0.8359119909644))),((((t501,(t524,t525)),(((t502,(t504,t505)),(t503,((t510,(t513,t514)),(t522,t523)))),((t511,t512),(t516,t517)))),((t506,(t515,(t520,t521))),((t507,t508),(t509,(t518,t519))))),(((t526,((t529,(t539,t540)),(t533,(t545,t546)))),(t528,((t530,(t547,t548)),((t531,t532)